# Capacitor de Placas Concéntricas Elípticas

En el capacitor, la placa exterior se encuentra a un potencial eléctrico $V_{ext}(\nu)$, mientras que la placa interior se encuentra a un potencial $V_{int}(\nu)$. El problema de valores en la frontera descrito en coordenadas elípticas se define:

$$\frac{1}{\sinh^2(\mu)+\sin^2(\nu)} (\frac{\partial^2V}{\partial\mu^2} + \frac{\partial^2V}{\partial\nu^2}) = 0,    \forall(\mu,\nu) \in \Omega$$

$$V(0.1, \nu) = \sin(2\nu), \forall\nu\in[0, 2\pi]$$
$$V(1, \nu) = 0, \forall\nu\in[0, 2\pi]$$

donde $\Omega$ es el dominio acotado por $\mu \in [0.1, 1] \times \nu \in [0, 2\pi]$.

In [189]:
import numpy as np
import sympy as sp
from sympy import init_printing, Eq
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

init_printing()

In [57]:
Vij, Vi1j, Vij1, Vim1j, Vijm1, du, dv = sp.symbols(r'V_{i\,j} V_{i+1\,j} V_{i\,j+1} V_{i-1\,j} V_{i\,j-1} \triangle\mu \triangle\nu')
u, v = sp.symbols(r'\mu \nu')
Vij, Vi1j, Vij1, Vim1j, Vijm1, du, dv, u, v

(V_{i,j}, V_{i+1,j}, V_{i,j+1}, V_{i-1,j}, V_{i,j-1}, \triangle\mu, \triangle\nu, \mu, \nu)

### Escribamos el esquema de diferencias finitas 

In [80]:
laplacian = (((Vi1j - 2*Vij + Vim1j)/du**2) + ((Vij1 - 2*Vij + Vijm1)/dv**2))
sys = Eq(laplacian, 0)
sp.simplify(sys)

            2                                                   2                                        
\triangle\mu ⋅(V_{i,j+1} + V_{i,j-1} - 2⋅V_{i,j}) + \triangle\nu ⋅(V_{i+1,j} - 2⋅V_{i,j} + V_{i-1,j})    
───────────────────────────────────────────────────────────────────────────────────────────────────── = 0
                                                 2             2                                         
                                     \triangle\mu ⋅\triangle\nu                                          

Véase que el termino senosoidal nunca es 0, luego basta con verificar que el término diferencial es 0

Finalmente si nombramos $$\frac{(\triangle\mu)^2}{(\triangle\nu)^2} = \alpha$$

In [95]:
alpha = sp.Symbol(r'\alpha')
rhs = Vi1j - 2*Vij + Vim1j + (Vij1 - 2*Vij + Vijm1)*alpha
new_eq = sp.Eq(rhs, 0)
new_eq

El cual puede ser escrito como:

In [98]:
rhs = -2*(1+alpha)*Vij + Vi1j + Vim1j + alpha*Vij1 + alpha*Vijm1
final_eq = Eq(rhs, 0)
final_eq

Ahora con nuestro esquema de diferencias finitas listas y las condiciones de frontera armemos nuestro sistema global teniendo además en cuenta que:
$$V_{i, j-1} = V_{i, j_{max}} \land V_{i, j_{max}+1} = V_{i, 0}$$

## Solución del Potencial Eléctrico

In [234]:
def electric_potential(n_u1, n_u2, v_ext, v_int):
    # Divide the domain acording to n_u1 and n_u2
    
    mu = np.linspace(0.1, 1, n_u1)
    nu = np.linspace(0, 2*np.pi, n_u2 + 1)
    dmu = mu[1] - mu[0]
    dnu = nu[1] - nu[0]
    alpha = (dmu**2)/(dnu**2)
    tg = lambda i, j: i*(n_u2) + j # To global
    dim = n_u1 * n_u2
    
    A = lil_matrix((dim, dim))
    b = lil_matrix((dim, 1))
    
    ij = np.zeros((dim, 2))
    
    j_max = n_u2 - 1
    
    for k in range(dim):
        # First let's identify i, j in the global system
        i = k//n_u2
        j = k - (k//n_u2)*n_u2
        ij[k, 0] = i
        ij[k, 1] = j
        
        ### Condiciones de frontera
        if(i == 0): # Condición interna
            nu = dnu * j # + nu_inicial, pero nu inicial es 0
            A[k, k] = 1
            b[k, :] = v_int(nu)
        elif(i == n_u1 - 1): # Condición externa
            A[k, k] = 1
            b[k, :] = v_ext
        else: # Condición general, aplicar ecuación
            A[k, k] = -2*(1+alpha)
            A[k, tg(i+1, j)] = 1
            A[k, tg(i-1, j)] = 1
            
            if(j == j_max):
                A[k, tg(i, 0)] = alpha
            else:
                A[k, tg(i, j+1)] = alpha
            
            if(j == 0):
                A[k, tg(i, j_max)] = alpha
            else:
                A[k, tg(i, j-1)] = alpha
            
    
    potential = spsolve(csr_matrix(A), csr_matrix(b))
    potential = potential.reshape(-1, 1)
        
    return np.hstack((ij, potential))

## Solución Campo Eléctrico

De electromagnetismo conocemos que para calcular el campo eléctrico podemos uilizar la expresión: 
$$\vec{E} = -\nabla V$$
Es decir:
$$\vec{E} = -\frac{1}{c \sqrt{\sinh^2{\mu} + \sin^2{\nu}}}(\frac{\partial V}{\partial \mu}, \frac{\partial V}{\partial \nu})$$
Como ya conocemos el potencial eléctrico en todos los lugares del espacio podemos solucionar con un esquema de diferencias finitas con la siguiente regla de formación: 
$$i = 0 \Rightarrow \frac{\partial V}{\partial \mu} = \frac{V_{i+1,j} - V_{i,j}}{\triangle\mu}$$
$$i = i_{max} \Rightarrow \frac{\partial V}{\partial \mu} = \frac{V_{i,j} - V_{i-1,j}}{\triangle\mu}$$
$$i \neq 0 \land i \neq i_{max} \Rightarrow \frac{\partial V}{\partial \mu} = \frac{V_{i+1,j} - V_{i-1,j}}{2\triangle\mu}$$
$$\forall j \in [0, j_{max}] \Rightarrow \frac{\partial V}{\partial \nu} = \frac{V_{i,j + 1} - V_{i,j - 1}}{2\triangle\nu}$$ 
Teniendo en cuenta que si $j=0$:
$$V_{i, j-1} = V_{i, j_{max}}$$
y si $j=j_{max}$:
$$V_{i, j_{max}+1} = V_{i, 0}$$

In [254]:
def electric_field(n_u1, n_u2, e_potential):
    mu = np.linspace(0.1, 1, n_u1)
    nu = np.linspace(0, 2*np.pi, n_u2 + 1)
    dmu = mu[1] - mu[0]
    dnu = nu[1] - nu[0]
    dim = n_u1 * n_u2
    e_field = np.zeros((dim, 4))
    e_field[:, 0:2] = e_potential[:, 0:2]
    tg = lambda i, j: i*(n_u2) + j # To global
    i_max = n_u1 - 1
    j_max = n_u2 - 1
    V = e_potential[:, 2]
    c = 1
    
    for k in range(dim):
        i = int(e_potential[k, 0])
        j = int(e_potential[k, 1])
        
        sf = 1 / (c*np.sqrt(np.sinh(mu[i])**2+np.sin(nu[j])**2))
        
        if(i==0):
            e_field[k, 2] = ((V[tg(i+1, j)] - V[tg(i, j)])/dmu) * sf
        elif(i == i_max):
            e_field[k, 2] = ((V[tg(i, j)] - V[tg(i-1, j)])/dmu) * sf
        else:
            e_field[k, 2] = ((V[tg(i+1, j)] - V[tg(i-1, j)])/(2*dmu)) * sf
        
        
        if(j==0):
            e_field[k, 3] = ((V[tg(i, j+1)] - V[tg(i, j_max)])/(2*dnu)) * sf
        elif(j==j_max):
            e_field[k, 3] = ((V[tg(i, 0)] - V[tg(i, j-1)])/(2*dnu)) * sf
        else:
            e_field[k, 3] = ((V[tg(i, j+1)] - V[tg(i, j-1)])/(2*dnu)) * sf
        
        
    return e_field

In [257]:
def to_eliptical(index_matrix, div_i):
    eliptical_coord = np.zeros((index_matrix.shape[0], index_matrix.shape[1]))
    my_i_index = index_matrix[:, 0]
    elipt_i_index = np.linspace(0.1, 1, div_i)
    my_elipt_i = elipt_i_index[my_i_index.astype(int)]
    
    for i in range(index_matrix.shape[0]):
        eliptical_coord[i,0] =  my_elipt_i[i]
        eliptical_coord[i,1] = (index_matrix[i,1]*2*np.pi)/(index_matrix[-1,1])
        eliptical_coord[i,2] = index_matrix[i,2]
    return eliptical_coord

In [ ]:
def plot_electric_solution(n_u1, n_u2, e_potential, e_field):
    e_potential = to_eliptical(e_potential, n_u1) 
    e_field =  to_eliptical(e_field, n_u1)
    
    # Take 

In [261]:
n_u = 100
n_v = 100
v_ext = 3
v_int = lambda v: np.sin(2*v)

e_potential = electric_potential(n_u, n_v, v_ext, v_int)
e_field = electric_field(n_u, n_v, e_potential)

transform = to_eliptical(e_potential, n_u)
transform

array([[0.1       , 0.        , 0.        ],
       [0.1       , 0.06346652, 0.12533323],
       [0.1       , 0.12693304, 0.24868989],
       ...,
       [1.        , 6.15625227, 3.        ],
       [1.        , 6.21971879, 3.        ],
       [1.        , 6.28318531, 3.        ]])